In [ ]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import pandas as pd

# Replace this with the correct path to your model directory in Colab
# or the model ID from Hugging Face Hub
model_dir = "C:/Users/DELL/OneDrive/Desktop/bert-agnews"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
device = 0 if torch.cuda.is_available() else -1
nlp=pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)
LABELS = ["World", "Sports", "Business", "Sci/Tech"]

st.set_page_config(page_title="BERT AG News Classifier", layout="centered")

st.title("BERT News Topic Classifier")
headline = st.text_area("Enter a news headline", height=100)

if st.button("Classify"):
    if headline.strip() == "":
        st.warning("Please enter a headline.")
    else:
        with st.spinner("Classifying..."):
            preds = nlp(headline)[0]
            rows = []
            for item in preds:
                idx = int(item["label"].split("_")[1]) if item["label"].startswith("LABEL_") else int(item["label"])
                rows.append({"label": LABELS[idx], "score": float(item["score"])})
            df = pd.DataFrame(rows).sort_values("score", ascending=False)
            st.write("**Predicted:**", df.iloc[0]["label"])
            st.table(df)